In [ ]:
!nvidia-smi

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
!pip install roboflow torch torchvision torchaudio ultralytics python-dotenv --quiet

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import os
import torch
from roboflow import Roboflow
from ultralytics import YOLO
from pathlib import Path
from google.colab import drive
from dotenv import load_dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ROOT_DIR = Path('/content/drive/MyDrive/tennis-cv/').resolve()

In [ ]:
TRAINING_DIR = ROOT_DIR / 'training'
MODELS_DIR = ROOT_DIR / 'models'

DATA_DIR = ROOT_DIR / 'data'
TENNIS_BALL_DIR = DATA_DIR / 'tennis_ball'
KEYPOINTS_DIR = DATA_DIR / 'keypoints'

DATA_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

TRAINING_DIR.mkdir(parents=True, exist_ok=True)
TENNIS_BALL_DIR.mkdir(parents=True, exist_ok=True)
KEYPOINTS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
if not any(TENNIS_BALL_DIR.iterdir()):
    print(f"No data found in {TENNIS_BALL_DIR}, downloading dataset...")
    load_dotenv(ROOT_DIR / '.env')
    rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
    project = rf.workspace("ry4ntr1").project("tennis-ball-m7zvw")
    version = project.version(3)
    dataset = version.download(model_format="yolov8", location=str(TENNIS_BALL_DIR), overwrite=True)
else:
    print(f"Data already present in {TENNIS_BALL_DIR}, no need to download.")

In [ ]:
os.chdir(str(ROOT_DIR))

In [ ]:
if os.path.exists(str(MODELS_DIR / 'best.pt')):
    trained_model = YOLO(str(MODELS_DIR / 'best.pt'))
else:
    model = YOLO('yolov8n.pt')
    model.train(data=str(TENNIS_BALL_DIR / 'data.yaml'), save=True, save_dir=MODELS_DIR, save_period=10, verbose=True)

In [ ]:
trained_model = YOLO('/content/runs/detect/train/weights/best.pt').to(device)
# trained_model = YOLO(str(MODELS_DIR / 'best.pt')).to(device)
sample_image_path = ROOT_DIR / 'sample_data/sample.png'
sample_vid_path = ROOT_DIR / 'sample_data/sample.mp4'

## Prediction On Sample Data

In [ ]:
trained_model.predict(task='detect', source=str(sample_image_path), save=True, save_dir=str('/content/runs/predict'))

In [ ]:
trained_model.predict(task='detect', source=str(sample_vid_path), save=True, save_dir=str('/content/runs/predict'))